In [1]:
from math import log, sqrt, exp
import time
import datetime as dt  
from datetime import datetime           # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr
from scipy.stats import norm  # normal cdf
import csv 
import statsmodels.formula.api as smf # for linear regression
from ib_insync import *
util.startLoop()

In [3]:
try:
    ib.disconnect()
    time.sleep(5)
except:
    print("no ib connection to disconnect")

# initialize connection to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)  # IB Trader Workstation

<IB connected to 127.0.0.1:7497 clientId=12>

In [4]:
#find the future contract
future_date_string='20221019'
future = Future('VIX',future_date_string,'CFE')
ib.qualifyContracts(future)
ib.reqContractDetails(future)

[ContractDetails(contract=Contract(secType='FUT', conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', exchange='CFE', currency='USD', localSymbol='VXV2', tradingClass='VX'), marketName='VX', minTick=0.05, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,LTH,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,OPENCLOSE,PEGBENCH,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='CFE', priceMagnifier=1, underConId=13455763, longName='CBOE Volatility Index', contractMonth='202210', industry='Indices', category='Volatility Index', subcategory='*', timeZoneId='US/Central', tradingHours='20221003:1700-20221004:1600;20221004:1700-20221005:1600;20221005:1700-20221006:1600;20221006:1700-20221007:1600;20221008:CLOSED;20221009:1700-20221010:1600', liquidHours='20221004:0830-20221004:1515;20221005:0830-20221005:1515;20221006

In [5]:
#get the price data for the vix future
ib.reqMarketDataType(4)
[Ticker]=ib.reqTickers(future)
Ticker
vix_1=Ticker.last

In [6]:
vix_1

29.1

In [7]:
#get the price of the VIX index
index = Index('VIX')
ib.qualifyContracts(index)
ib.reqContractDetails(index)
[Ticker]=ib.reqTickers(index)
Ticker
vix_spot=Ticker.close

In [8]:
vix_spot

30.1

In [32]:
#construct the signal for backwardation/contango
b_signal=(vix_1/vix_spot)-1
b_signal
#if b_signal is positive, vix term structure is in contango
#if b_signal is negative, vix term structure is in bakwardation

-0.03322259136212624

In [33]:
#remaining business days
today=dt.date.today()
future_date=datetime.strptime(future.lastTradeDateOrContractMonth, "%Y%m%d").date()
days = np.busday_count(today, future_date)
days
#daily roll calculation
daily_roll=(vix_1-vix_spot)/days
daily_roll

-0.018867924528301886

In [34]:
order=()
#first trade:
if b_signal<0 and daily_roll<-0.10: 
    order = MarketOrder('BUY', 10)
elif b_signal>0 and daily_roll>0.10:
    order = MarketOrder('SELL', 10)
order

()

In [39]:
#send order to ib
trade = ib.placeOrder(future, order)

In [12]:
positions = [p for p in ib.positions() if p.contract.symbol == "VIX"]
future_pos = [p for p in positions if p.contract.secType == "FUT"]
future_pos

[Position(account='DU6067498', contract=Future(conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', currency='USD', localSymbol='VXV2', tradingClass='VX'), position=10.0, avgCost=29452.38)]

In [15]:
f.contract.lastTradeDateOrContractMonth

'20221019'

In [14]:
#exit strategy
for f in future_pos:
    days = np.busday_count(today, datetime.strptime(f.contract.lastTradeDateOrContractMonth, "%Y%m%d").date())
    daily_roll=(vix_1-vix_spot)/days #need to find a way to update the price here
    if f.position>0 and (days<=9 or daily_roll>-0.05):
        order = MarketOrder('SELL', f.position)
        trade = ib.placeOrder(future, order)
    elif f.position<0 and (days<=9 or daily_roll<0.05):
        order = MarketOrder('BUY', abs(f.position))
        trade = ib.placeOrder(future, order) 

In [17]:
## Q4- E-mini and VIX Futures Historical Data
today=dt.date.today()
today_str=today.strftime("%m/%d/%Y")
a_year_ago = today - dt.timedelta(days=365)
a_year_ago_str = a_year_ago.strftime("%m/%d/%Y")
#data from marketwatch for vx00 and es00
vix_futures_data = pd.read_csv('https://www.marketwatch.com/investing/future/vx00/downloaddatapartial?startdate='+a_year_ago_str+'%2000:00:00&enddate='+today_str+'%2000:00:00&daterange=d30&frequency=p1d&csvdownload=true&downloadpartial=false&newdates=false')
sp500_futures=pd.read_csv("https://www.marketwatch.com/investing/future/es00/downloaddatapartial?startdate="+a_year_ago_str+"%2000:00:00&enddate="+today_str+"%2000:00:00&daterange=d30&frequency=p1d&csvdownload=true&downloadpartial=false&newdates=false")
#use open prices to calculate the percentage changes
sample = pd.DataFrame(columns=["Date","sp_mini_change","vix_futures_change"])
sample["Date"]=vix_futures_data["Date"]
sample["sp_mini_change"]=sp500_futures['Open'].str.replace(',', '').astype(float)/sp500_futures['Open'].str.replace(',', '').astype(float).shift(periods=-1)-1
sample["vix_futures_change"]=vix_futures_data['Open']/vix_futures_data['Open'].shift(periods=-1)-1
sample.head(10)

,Date,sp_mini_change,vix_futures_change
0,10/03/2022,-0.017298,0.022617
1,09/30/2022,-0.019771,0.036851
2,09/29/2022,0.016832,-0.038647
3,09/28/2022,0.000136,0.006483
4,09/27/2022,-0.010053,0.024917
5,09/26/2022,-0.019061,0.082734
6,09/23/2022,-0.004415,-0.021127
7,09/22/2022,-0.021850,0.038012
8,09/21/2022,-0.011280,0.032453
9,09/20/2022,0.008613,-0.025735


In [18]:
#chunk for change regression
model=smf.ols("vix_futures_change~sp_mini_change",sample).fit()
# get the model beta
beta=model.params.values[1]
print("optimal hedge ratio is "+str(beta))

optimal hedge ratio is -3.103733647364325


In [19]:
#find the sp500 mini future
#future_date_string needs to be checked
future_date_string='20221216'
future = Future('ES', future_date_string,'GLOBEX')
#ib.qualifyContracts(future)
ib.reqContractDetails(future)[0]
#How to get the price data for the vix future
ib.reqMarketDataType(4)
[Ticker]=ib.reqTickers(future)
Ticker
sp500mini_price=Ticker.last
print('sp500mini price is '+str(sp500mini_price))

sp500mini price is 3786.0


In [27]:
#Optimal Hedge amount

hedge_amount=(beta*(vix_1*1000)/(sp500mini_price*50))
hedge_amount

-0.4771191185330262

In [28]:
f.position

10.0

In [29]:
if hedge_amount<0:
    hedge_order = MarketOrder('SELL', round(abs(hedge_amount)*f.position)) #be careful on that f.position if you have more than one contract
else:
    hedge_order = MarketOrder('BUY',round(abs(hedge_amount)*f.position))
hedge_order

MarketOrder(action='SELL', totalQuantity=5)

In [53]:
#send order to ibkr
trade = ib.placeOrder(future, hedge_order)

Error 321, reqId 63: Error validating request.-'bN' : cause - Order size does not conform to market rule.
Canceled order: Trade(contract=Future(symbol='ES', lastTradeDateOrContractMonth='20221216', exchange='GLOBEX'), order=MarketOrder(orderId=63, clientId=10, action='SELL', totalQuantity=-5), orderStatus=OrderStatus(orderId=63, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 10, 3, 17, 54, 17, 580650, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 10, 3, 17, 54, 17, 593546, tzinfo=datetime.timezone.utc), status='Cancelled', message="Error 321, reqId 63: Error validating request.-'bN' : cause - Order size does not conform to market rule.", errorCode=321)], advancedError='')
Peer closed connection.
